In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth
from folium import GeoJson
import geopandas as gpd

import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('../bin/data_def.csv')

df.head()

,provincia,link_coche,fecha_extraccion,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carroceria,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion
0,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles VOLKSWAGEN Polo (2019),Matriculado: 10/2019,93000,COCHES AMB,Profesional,C,12 meses,20950,29044,4053 mm Largo,1751 mm Ancho,1461 mm Alto,Capacidad del maletero (l)355/1.125,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.548,Peso/Masa max. autorizado (kg)1.855,Número de puertas5,"Consumo medio 5,80 litros/100 km","Consumo en carretera 4,90 litros/100 km","Consumo urbano 7,70 litros/100 km",CO2 (g/km) 138 g/km,"Depósito de combustible (l) 40,00 litros",Combustible Gasolina,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.984,Sobrealimentación Turbo,Tipo de transmisión Automática secuencial,TracciónTracción delantera,Número de marchas6,Potencia (KW)147,Potencia (CV)200,Par (Nm)320,Velocidad máxima (km/h)237,"Aceleración 0-100 km/h (s)6,70"
1,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles OPEL Corsa (2009),Matriculado: 07/2009,236945,AUTOS CAPRICAR,Profesional,B,No,4000,15660,3999 mm Largo,1737 mm Ancho,1488 mm Alto,Capacidad del maletero (l)285/1.100,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.511,Peso/Masa max. autorizado (kg)1.695,Número de puertas5,"Consumo medio 4,60 litros/100 km","Consumo en carretera 3,80 litros/100 km","Consumo urbano 5,90 litros/100 km",CO2 (g/km) 127 g/km,"Depósito de combustible (l) 45,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.248,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)66,Potencia (CV)90,Par (Nm)200,Velocidad máxima (km/h)172,"Aceleración 0-100 km/h (s)12,70"
2,Murcia,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles SKODA Octavia (2017),Matriculado: 08/2017,44300,SKOMOVIL MURCIA,Profesional,ECO,12 meses,15900,25030,4670 mm Largo,1814 mm Ancho,1458 mm Alto,Capacidad del maletero (l)460/1.450,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.680,Peso/Masa max. autorizado (kg)1.878,Número de puertas5,"Consumo medio 5,70 litros/100 km","Consumo en carretera 4,60 litros/100 km","Consumo urbano 7,40 litros/100 km",CO2 (g/km) 101 g/km,"Depósito de combustible (l) 50,00 litros",Combustible Gasolina/gas,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.395,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)81,Potencia (CV)110,Par (Nm)200,Velocidad máxima (km/h)195,"Aceleración 0-100 km/h (s)10,90"
3,Murcia,https://www.autocasion.com/coches-km0/km-0/sko...,25/11/2024,Detalles SKODA Kamiq (2024),Matriculado: 03/2024,10600,SKOMOVIL MURCIA,Profesional,C,12 meses,24900,no disponible,NaN,NaN,NaN,NaN,"4x4, SUV o pickup",NaN,NaN,NaN,5 Puertas,NaN,NaN,NaN,NaN,NaN,Gasolina,NaN,NaN,NaN,Automático,NaN,NaN,NaN,110,NaN,NaN,NaN
4,Murcia,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles SKODA Octavia (2024),Matriculado: 02/2024,13673,SKOMOVIL MURCIA,Profesional,0 EMISIONES,12 meses,33500,no disponible,NaN,NaN,NaN,NaN,Familiar,NaN,NaN,NaN,5 Puertas,NaN,NaN,NaN,NaN,NaN,Eléctrico,NaN,NaN,NaN,Automático,NaN,NaN,NaN,204,NaN,NaN,NaN


## Limpieza preliminar

In [5]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
24,CO2,216,19.030232,36.921279,object
37,aceleracion,138,17.555134,17.555134,object
36,velocidad_max,140,17.540529,17.540529,object
35,par,212,17.540529,17.540529,object
15,capacidad_maletero,875,17.540529,17.540529,object
18,batalla,325,17.540529,17.540529,object
33,potencia_kw,226,17.379874,17.379874,object
14,alto,413,17.365269,17.365269,object
13,ancho,287,17.365269,17.365269,object
12,largo,671,17.365269,17.365269,object


In [7]:
df['tipo_cambio'].unique()

#Hay nans que se toman como floats. Lo convierto a "no disponible"
df['tipo_cambio'] = df['tipo_cambio'].fillna("no disponible")

#No sé qué es manual automatizada, si queremos que sea manual hay que poner la lambda al revés
df['tipo_cambio'] = df['tipo_cambio'].apply(lambda x: 'Manual' if 'Manual' in x else 'Automático')

In [9]:
df["carroceria"] = df["carroceria"].str.replace("Carrocería","")

df["combustible"] = df["combustible"].str.replace("combustible ","").str.replace("Combustible ","")

In [11]:
df['fecha_matriculacion'] = df['fecha_matriculacion'].str.split("Matriculado: ").str[1]

def limpiar_fecha(fecha):
    try:
        mes, ano = fecha.split('/')
        return int(mes), int(ano)
    except:
        return np.nan, int(fecha)

df[['mes_matriculacion', 'ano_matriculacion']] = df['fecha_matriculacion'].apply(lambda x: pd.Series(limpiar_fecha(x)))

df = df.drop(columns=['fecha_matriculacion'])

In [13]:
df['mes_matriculacion'] = df['mes_matriculacion'].fillna(0).astype('int64')
df['ano_matriculacion'] = df['ano_matriculacion'].fillna(0).astype('int64')

In [15]:
df = df[df["tipo_vendedor"] == "Profesional"]
df = df.drop(columns=['tipo_vendedor'])

In [17]:
df["garantia"] = df["garantia"].str.split(' ').str[0].replace("Sí",np.nan).replace("No",0).astype(float)

In [19]:
df["precio_contado"] = pd.to_numeric(df["precio_contado"], errors='coerce')

df["precio_nuevo"] = pd.to_numeric(df["precio_nuevo"], errors='coerce')

df['largo'] = pd.to_numeric(df['largo'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['ancho'] = pd.to_numeric(df['ancho'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['alto'] = pd.to_numeric(df['alto'].str.extract('(\d+\.?\d*)')[0], errors='coerce')

df['capacidad_maletero'] = pd.to_numeric(df['capacidad_maletero'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
df["num_plazas"] = df["num_plazas"].str.split("s").str[1].astype("Int64")
df["batalla"] = (df["batalla"].str.replace(r'Batalla \(mm\)\s*', '', regex=True).str.replace('.', '', regex=False))
df["batalla"] = pd.to_numeric(df["batalla"], errors='coerce').astype("Int64")
df["peso"] = df["peso"].str.replace('Peso/Masa max. autorizado (kg)', '').str.replace('.','').astype("Int64")
df["num_puertas"] = df["num_puertas"].str.extract(r'(\d+)').astype("Int64")
df["consumo_medio"] = df["consumo_medio"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_carretera"] = df["consumo_carretera"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_urbano"] = df["consumo_urbano"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["co2"] = df["CO2"].str.extract(r'(\d{1,3})(?= g/km)')[0].astype(float)
df["num_cilindros"] = df["cilindros"].str.extract(r'(\d+)').astype("Int64")
df["cilindrada"] = df["cilindrada"].str.extract(r'(\d+,\d+|\d+\.\d+)', expand=False).str.replace(',', '').str.replace('.', '').astype("Int64")
df["deposito"] = df["deposito"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

df["num_marchas"] = df["num_marchas"].str.extract(r'(\d+)').astype("Int64")
df["potencia_kw"] = df["potencia_kw"].str.extract(r'(\d+)').astype("Int64")
df["potencia_cv"] = df["potencia_cv"].str.extract(r'(\d+)').astype("Int64")
df["par"] = df["par"].str.extract(r'(\d+)').astype("Int64")
df["velocidad_max"] = df["velocidad_max"].str.extract(r'(\d+)').astype("Int64")
df["aceleracion"] = df["aceleracion"].str.extract(r'(\d+,\d+|\d+\.\d+)')[0].str.replace(",", ".").astype(float)
df['id_sobrealimentacion'] = df['sobrealimentacion'].apply(lambda x: x.replace('Sobrealimentación', '').replace('Tipo de sobrealimentador', 'no disponible').strip() if pd.notna(x) and x != '-' else 'no disponible')

In [23]:
df["traccion"].unique()

df["traccion"] = df['traccion'].fillna("no disponible")
df["traccion"] = df["traccion"].str.replace('Tracción', '').str.replace('Todo terreno tracción', '').apply(lambda x: x.strip())

In [27]:
marca_listado = pd.read_csv('../bin/listado_marcas.csv')

lista_marcas = [marca.upper() for marca in marca_listado['nombre_marca'].tolist()]

df['marca_modelo'] = df['marca_modelo'].fillna('').str.replace("Detalles ", "").str.strip().str.upper()

df['marca'] = df['marca_modelo'].apply(lambda x: next((marca for marca in lista_marcas if marca in x), np.nan))
df['modelo'] = df.apply(lambda x: re.sub(r'\s*\(.*?\)', '', x['marca_modelo'].replace(x['marca'], "").strip()).strip() if pd.notnull(x['marca']) else x['marca_modelo'], axis=1)

In [29]:
df = df.drop(columns=['marca_modelo', 'CO2', 'cilindros', 'sobrealimentacion', 'carroceria'])

In [31]:
df.rename(columns={'distintivo_ambiental': 'id_distintivo_ambiental'
                  , 'marca': 'id_marca'
                  , 'modelo': 'id_modelo'
                  , 'nombre_vendedor': 'id_concesionario'
                  , 'provincia': 'id_provincia'
                  , 'traccion': 'id_traccion'}, inplace=True)

In [33]:
df['combustible'] = (
    df['combustible']
    .str.replace('-', 'no disponible')
    .str.replace('Diésel', 'Diesel')
    .str.replace('Gasolina y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Diesel y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Corriente eléctrica', 'Eléctrico', regex=False)
    .str.replace(r'\bHíbrido\b(?!.*Enchufable)', 'Híbrido Enchufable', regex=True)
)

In [35]:
df['id_concesionario'] = df['id_concesionario'].apply(lambda x: x.split('-')[0])

In [37]:
for columna in df.columns:
    df[columna] = df[columna].replace(["no disponible", "<NA>", "-", "0", 0, np.nan], None)
    

In [39]:
df = df[df['precio_contado'].notna()]

## Rtdo limpieza

In [41]:
df.sample(10)

,id_provincia,link_coche,fecha_extraccion,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
5249,Madrid,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,136184,AUTOMOVILES CANALCAR,B,12.0,13990.0,30650.0,4454.0,1798.0,1545.0,420.0,5,2760,2030,5,5.2,4.7,6.1,61.0,Diesel,1995,Manual,trasera,6,105,143,320,200,9.6,11,2011,136.0,4,Turbo de geometría variable,BMW,X1
2044,Ciudad Real,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,109228,"VOLVO VYPSA, Volvo",C,12.0,29900.0,52400.0,4963.0,1890.0,1443.0,500.0,5,2941,2270,4,4.6,4.2,5.4,55.0,Diesel,1969,Automático,delantera,8,140,190,400,230,8.2,7,2020,121.0,4,Turbo de geometría variable,VOLVO,S90
6093,Tarragona,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,75359,GO MOTOR.,C,12.0,15990.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Gasolina,None,Manual,None,None,None,110,None,None,None,5,2021,None,None,None,SEAT,IBIZA
5820,Barcelona,https://www.autocasion.com/coches-km0/km-0/sea...,26/11/2024,25,GRUPO LESSEPS MOTORS,C,36.0,27900.0,32880.0,4381.0,1841.0,1601.0,510.0,5,2638,1870,5,None,None,None,50.0,Gasolina,None,Manual,delantera,6,81,110,200,180,11.3,3,2024,None,3,Turbo de geometría variable,SEAT,ATECA
4124,Madrid,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,56561,Autoplanet Leganés II,C,12.0,21500.0,30735.0,4489.0,1836.0,1607.0,472.0,5,2646,1920,5,5.4,4.8,7.0,55.0,Gasolina,1333,Automático,delantera,7,103,140,240,200,9.5,5,2021,122.0,4,Turbo,RENAULT,KADJAR
1009,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,80130,CLICARS MADRID,C,12.0,24190.0,41935.0,4360.0,1824.0,1526.0,470.0,5,2670,2115,5,4.4,4.0,4.9,51.0,Diesel,1995,Automático,delantera,8,110,150,350,207,10.0,12,2020,114.0,4,Turbo de geometría variable,BMW,X2
5516,Madrid,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,39493,SPOTICAR AUTO RALLYE,0 EMISIONES,48.0,29900.0,48850.0,4180.0,1800.0,1570.0,332.0,5,2600,2170,5,None,None,None,None,Eléctrico,None,Automático,delantera,1,150,204,395,167,7.9,5,2023,None,None,None,HYUNDAI,KONA
4112,Málaga,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,14150,Holamotor Málaga,C,12.0,18900.0,24668.0,4074.0,1751.0,1451.0,351.0,5,2548,1640,5,None,None,None,40.0,Gasolina,None,Manual,delantera,5,70,95,175,187,10.8,5,2023,None,3,Turbo,VOLKSWAGEN,POLO
3939,La Coruña,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,10661,"GONZACAR, S.L.",C,24.0,28750.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Gasolina,1000,Automático,None,None,None,155,None,None,None,10,2023,None,None,None,FORD,PUMA 1997
3735,Lugo,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,81000,Ditram Monforte | Foz | Automotor10,ECO,24.0,25900.0,None,None,None,None,None,None,None,1540,5,5.0,None,None,None,Híbrido Enchufable,1987,Automático,None,5,None,184,None,None,None,7,2021,None,None,None,LEXUS,UX


In [43]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
33,co2,207,56.208236,56.208236,object
19,consumo_urbano,142,56.161616,56.161616,object
18,consumo_carretera,82,56.161616,56.161616,object
17,consumo_medio,114,50.458430,50.458430,object
35,id_sobrealimentacion,5,34.079254,43.045843,object
22,cilindrada,199,28.189588,28.189588,object
20,deposito,81,24.413364,24.413364,object
34,num_cilindros,7,23.512044,46.511267,object
12,capacidad_maletero,359,19.906760,19.906760,object
29,velocidad_max,137,19.704740,19.704740,object


In [45]:
df.to_csv('../bin/data_preprocess.csv', index=False)

## Gráficos

In [ ]:
plt.figure(figsize=(10, 6))
for tipo_combustible in df['combustible'].unique():
    subset = df[df['combustible'] == tipo_combustible]
    plt.scatter(subset['potencia_cv'], subset['precio_contado'], label=tipo_combustible, alpha=0.6)

# Configurar etiquetas y título
plt.xlabel('Potencia (cv)')
plt.ylabel('Precio')
plt.title('Relación entre Precio y Potencia por Tipo de Combustible')
plt.legend(title='Combustible')
plt.yscale('log')

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_precio_potencia.png')

plt.show()



In [ ]:
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['kilometraje'], df['precio_contado'], c=df['ano_matriculacion'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Año de Matriculación')

# Configurar etiquetas y título
plt.xlabel('Kilometraje')
plt.ylabel('Precio Contado')
plt.title('Relación entre Kilometraje y Precio Contado por Año de Matriculación')
plt.yscale('log')

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_kilometraje_precio.png')

plt.show()



In [ ]:
precio_medio = df.groupby('id_marca')[['precio_nuevo', 'precio_contado']].mean().sort_values(by='precio_contado', ascending=False)

precio_medio.plot(kind='bar', figsize=(18, 7))
plt.xlabel('Marca')
plt.ylabel('Precio Medio')
plt.title('Comparación del Precio Nuevo y Precio Contado por Marca')
plt.xticks(rotation=45, fontsize=8)

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_precio_nuevo_contado.png')

plt.show()



In [ ]:
plt.figure(figsize=(20, 15))
df.boxplot(column='precio_contado', by='id_distintivo_ambiental', grid=False)
plt.xlabel('Distintivo Ambiental')
plt.ylabel('Precio Contado')
plt.title('Distribución del Precio Contado por Distintivo Ambiental')
plt.suptitle('')  # Elimina el título automático generado por boxplot
plt.yscale('log')

#Guardamos la imagen para usarla en Streamlit
plt.savefig('../bin/imagenes/grafico_precio_distintivo.png')

plt.show()



## Visualizaciones

In [ ]:
df.columns

In [ ]:
categorical_columns = ['id_provincia', 'id_concesionario', 'id_distintivo_ambiental','combustible','tipo_cambio', 'id_traccion', 'mes_matriculacion'
               , 'ano_matriculacion','id_sobrealimentacion', 'id_marca', 'id_modelo']
continuous_columns = []

for columna in df.columns: 
    if columna not in categorical_columns: 
        continuous_columns.append(columna)

In [ ]:
def metricas (df, columna):
    
    if columna in categorical_columns:
        
        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 2, 1)
        sns.countplot(data=df, x=columna)
        plt.title(f'Distribución de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(data=df, x=columna, y='precio_contado')
        plt.title(f'Boxplot de {columna} con precio')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()
    
    elif columna in continuous_columns:

        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 3, 1)
        sns.histplot(df[columna], kde=True)
        plt.title(f'Histograma de {columna}')
        
        
        plt.subplot(1, 3, 2)
        sns.boxplot(x=df[columna])
        plt.title(f'Boxplot de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 3, 3)
        sns.scatterplot(x=df[columna], y=df['precio_contado'])
        plt.title(f'Scatterplot de {columna} con precio')
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()

In [ ]:
for columna in continuous_columns:
    metricas(df, columna)

In [ ]:
for columna in categorical_columns:
    metricas(df, columna)

In [ ]:
correlation_matrix = df[continuous_columns].corr()

plt.figure(figsize=(25, 15))

sns.heatmap(correlation_matrix, annot=True)

plt.title('Mapa de Calor de Correlaciones')
plt.show()